In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from scipy.integrate import odeint
from tqdm import tqdm
import ipywidgets as widgets
import sympy as sym
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [5]:
#Set the seed
np.random.seed(42)

results_path = 'results'
params_path = 'params'
main_path = os.path.normpath(os.getcwd() + os.sep + os.pardir)

#Define the parameters of the model
num_species = 4
num_scenarios = 20
mu_H = 5.5*10**-5 #TAKIMOTO'S VALUE
lambda_H = 1.3*10**-5 #TAKIMOTO'S VALUE
lambda_V = np.random.uniform(0.35, 0.45)*100000 #TAKIMOTO'S VALUE
gamma = 0.0035 #TAKIMOTO'S VALUE
tau_HV = np.random.uniform(0.020, 0.025) #TAKIMOTO'S VALUE
tau_VH = np.random.uniform(0.20, 0.28) #TAKIMOTO'S VALUE 
mu_V = np.random.uniform(0.11, 0.15) #TAKIMOTO'S VALUE
sigma_V = np.random.uniform(0.25, 0.36) #TAKIMOTO'S VALUE
sigma_H = np.random.uniform(3.9, 4.5) #TAKIMOTO'S VALUE

dict_type = {0: 'domi', 1: 'high', 2: 'low'}

#NO SE SI ESTO VA ACA O EN OTRA CARPETA
vecTime = np.linspace(0, 500, 100)
H_total = 1
p_H_Infected = 0.09
V_total = np.random.uniform(10000,50000, size=num_species)

p_V_Infected = np.random.uniform(0, 0.15, size=num_species)

condInit = [H_total*(1-p_H_Infected), H_total*(p_H_Infected)]

for i in range(num_species):
    condInit.append(V_total[i]*(1-p_V_Infected[i]))

for i in range(num_species):
    condInit.append(V_total[i]*(p_V_Infected[i]))

b = np.array([lambda_V/mu_V for i in range(num_species)])
A = np.zeros((num_species,num_species))

mat_a_temp = np.load(os.path.join(main_path, params_path, str(num_species), 'domi'+"_"+'domi', 'mat_a_'+str(13)+'.npy'))


for i in range(num_species):
    for j in range(num_species):
        if i == j:
            A[i,j] = 1
        else:
            A[i,j] = mat_a_temp[i,j]*lambda_V/mu_V

x_bar = np.linalg.solve(A, b)


[-2.07722996  2.09733761  2.71714935  3.44253716]


In [6]:
#Define additional functions
def b_i(V, H, mat_pi):
    return (sigma_V[i]*V*sigma_H[i]*H)/(sigma_V[i]*V + sigma_H[i]*H + np.sum([mat_pi[i,j]*sigma_V[j]*V for j in range(num_species)]))

#Define the model function
def model_HostVectorNspecies(variables, t, no_epi, mat_a, mat_pi, lambda_H, lambda_V, gamma, tau_HV, tau_VH, mu_H, mu_V):
    #The first 2 variables are the host population (humans, S-I)
    H_S = variables[0]
    H_I = variables[1]
    #The next 2*num_species variables are the vector population of different species (mosquitoes, S-I)
    V_S = [variables[i]  for i in range(2, 2+num_species)]
    V_I = [variables[i]  for i in range(2+num_species, 2+2*num_species)]

    
    derivatives = list()

    if no_epi:
        dH_Sdt = lambda_H - mu_H*H_S 
        dH_Idt = - mu_H*H_I - gamma*H_I 
        derivatives = derivatives + [dH_Sdt, dH_Idt]
    
        for i in range(num_species):
            dV_Sidt = -mu_V*V_S[i] + lambda_V*(1-np.sum([mat_a[i,j]*(V_S[j]+V_I[j]) for j in range(num_species)]))
            derivatives.append(dV_Sidt)
            
        for i in range(num_species):
            dV_Iidt = -mu_V*V_I[i]
            derivatives.append(dV_Iidt)
    else:
        dH_Sdt = lambda_H - mu_H*H_S + gamma*H_I - np.sum([tau_HV*(V_I[i])/(V_S[i]+V_I[i])*(b_i(V_S[i]+V_I[i], H_S+H_I, mat_pi)*H_S)/(H_S+H_I) for i in range(num_species)])
        dH_Idt = - mu_H*H_I - gamma*H_I + np.sum([tau_HV*(V_I[i])/(V_S[i]+V_I[i])*(b_i(V_S[i]+V_I[i], H_S+H_I, mat_pi)*H_S)/(H_S+H_I) for i in range(num_species)])
        
        derivatives = derivatives + [dH_Sdt, dH_Idt]
    
        for i in range(num_species):
            dV_Sidt = -mu_V*V_S[i] - tau_VH*(H_I)/(H_S+H_I)*(b_i(V_S[i]+V_I[i], H_S+H_I, mat_pi)*V_S[i])/(V_S[i] + V_I[i]) + lambda_V[i]*(1-np.sum([mat_a[i,j]*(V_S[j]+V_I[j]) for j in range(num_species)]))
            derivatives.append(dV_Sidt)
            
        for i in range(num_species):
            dV_Iidt = -mu_V[i]*V_I[i] + tau_VH[i]*(H_I)/(H_S+H_I)*(b_i(V_S[i]+V_I[i], H_S+H_I, mat_pi)*V_S[i])/(V_S[i] + V_I[i]) 
            derivatives.append(dV_Iidt)
    

    return derivatives


In [8]:
mat_zeros = np.zeros((num_species, num_species))
lambda_V_0 = np.zeros(num_species)
tau_0 = np.zeros(num_species)

def plot_same_scenarios(scenario1, scenario2, no_epi, lambda_V):
    mat_a_temp = np.load(os.path.join(main_path, params_path, str(num_species), scenario1+"_"+scenario2, 'mat_a_'+str(13)+'.npy'))
    mat_pi_temp =  np.load(os.path.join(main_path, params_path, str(num_species), scenario1+"_"+scenario2, 'mat_pi_'+str(13)+'.npy'))
    
    simu_test = odeint(model_HostVectorNspecies, condInit, vecTime, args=(no_epi,mat_a_temp, mat_pi_temp, lambda_H, lambda_V, gamma, tau_HV, tau_VH, mu_H, mu_V))

    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(scenario1+"_"+scenario2)
    
    fig.subplots_adjust(left=0.25, bottom=0.25)
    
    axes[0,0].plot(vecTime, simu_test[:,0], label="H_s")
    axes[0,0].plot(vecTime, simu_test[:,1], label="H_i")
    axes[0,0].set_title('Host')
    axes[0,0].set_ylabel('H')
    axes[0,0].grid()
    axes[0,0].set_xlabel('Time')

    axes[0,1].plot(vecTime, simu_test[:,2], label="V_s1")
    axes[0,1].plot(vecTime, simu_test[:,-4], label="V_h1")
    axes[0,1].set_title('Vector 1')
    axes[0,1].set_ylabel('V')
    axes[0,1].grid()
    axes[0,1].set_xlabel('Time')

    axes[0,2].plot(vecTime, simu_test[:,3], label="V_s2")
    axes[0,2].plot(vecTime, simu_test[:,-3], label="V_h2")
    axes[0,2].set_title('Vector 2')
    axes[0,2].set_ylabel('V')
    axes[0,2].grid()
    axes[0,2].set_xlabel('Time')

    axes[1,0].plot(vecTime, simu_test[:,4], label="V_s3")
    axes[1,0].plot(vecTime, simu_test[:,-2], label="V_h3")
    axes[1,0].set_title('Vector 3')
    axes[1,0].set_ylabel('V')
    axes[1,0].grid()
    axes[1,0].set_xlabel('Time')

    axes[1,1].plot(vecTime, simu_test[:,5], label="V_s4")
    axes[1,1].plot(vecTime, simu_test[:,-1], label="V_h4")
    axes[1,1].set_title('Vector 4')
    axes[1,1].set_ylabel('V')
    axes[1,1].grid()
    axes[1,1].set_xlabel('Time')

    axes[1,2].axis('off')


interact(plot_same_scenarios, scenario1=['domi','high','low'], scenario2=['domi','high','low'], no_epi=[True, False], 
         lambda_V=(10,1200,100))
    

interactive(children=(Dropdown(description='scenario1', options=('domi', 'high', 'low'), value='domi'), Dropdo…

<function __main__.plot_same_scenarios(scenario1, scenario2, no_epi, lambda_V)>